# 1. Import Dependencies

In [ ]:
# !pip install opencv-python

In [2]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

# Import shutil
import shutil

In [4]:
BACKUP_DIR = r'C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\BBOX\BackUp'
DATASET_DIR = r"C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Dataset"
ROOT_DIR = r"C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\Models\TFODCourse\Tensorflow"
IMAGES_PATH = os.path.join(ROOT_DIR, "workspace", "images", "collectedimages")
LABELIMG_PATH = os.path.join(ROOT_DIR, 'labelimg')

# 2. Define Images to Collect

In [6]:
labels = os.listdir(BACKUP_DIR)
number_imgs = 30

# 3. Setup Folders 

In [7]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
        !mkdir {IMAGES_PATH}
#for each species in labels
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

In [8]:
### moveback
source_folders = os.listdir(IMAGES_PATH)
#move all files within the source folders to the same folder in DATASET_DIR
for folder in source_folders:
    for img_name in os.listdir(os.path.join(IMAGES_PATH, folder)):
        # print(os.path.join(IMAGES_PATH, folder, img_name))
        # if the file not existed at destination
        if not os.path.exists(os.path.join(DATASET_DIR, folder, img_name)):
            shutil.copy(os.path.join(IMAGES_PATH, folder, img_name), os.path.join(DATASET_DIR, folder, img_name))

In [46]:
### Transfer images
def filter():
    folder_list = os.listdir(BACKUP_DIR)
    for folder in folder_list:
        #Check content of files in folder, if the file is empty then remove that file 
        file_list = os.listdir(os.path.join(BACKUP_DIR, folder))
        for file in file_list:
            if os.stat(BACKUP_DIR + '\\' + folder + '\\' + file).st_size == 0:
                os.remove(BACKUP_DIR + '\\' + folder + '\\' + file)
                print('Removed empty file: ' + file)
            if file.split('_')[0] != folder:
                os.remove(BACKUP_DIR + '\\' + folder + '\\' + file)
                print('File not compatible: ' + file)

def transfer_imgs():
    filter()
    for annotation_folder in labels:
        txt_path = os.listdir(os.path.join(BACKUP_DIR, annotation_folder))
        imgs_name = [(txt_file.split('.')[0]+'.jpg') for txt_file in txt_path if txt_file.endswith('.txt')]
        for img_name in imgs_name:
            destination = os.path.join(IMAGES_PATH, annotation_folder, img_name)
            if not os.path.exists(destination):
                shutil.copy(os.path.join(DATASET_DIR, annotation_folder, img_name), destination)

transfer_imgs()

# 4. Capture Images

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

# 5. Image Labelling

In [ ]:
# !pip install --upgrade pyqt5 lxml

In [48]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

Cloning into 'Tensorflow\labelimg'...


In [49]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [5]:
!cd {LABELIMG_PATH} && python labelImg.py

Cancel creation.
Image:C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Dataset\Eriovixia-laglaizei\Eriovixia-laglaizei_1.jpg -> Annotation:C:/Code/ChungyuanProjects/SpiderDataset01/inaturalist/Dataset/Eriovixia-laglaizei\Eriovixia-laglaizei_1.xml
Image:C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Dataset\Eriovixia-laglaizei\Eriovixia-laglaizei_2.jpg -> Annotation:C:/Code/ChungyuanProjects/SpiderDataset01/inaturalist/Dataset/Eriovixia-laglaizei\Eriovixia-laglaizei_2.xml
Image:C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Dataset\Eriovixia-laglaizei\Eriovixia-laglaizei_3.jpg -> Annotation:C:/Code/ChungyuanProjects/SpiderDataset01/inaturalist/Dataset/Eriovixia-laglaizei\Eriovixia-laglaizei_3.xml
Image:C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Dataset\Eriovixia-laglaizei\Eriovixia-laglaizei_4.jpg -> Annotation:C:/Code/ChungyuanProjects/SpiderDataset01/inaturalist/Dataset/Eriovixia-laglaizei\Eriovixia-laglaizei_4.xml
Image:C:\Code\ChungyuanProjects\SpiderD

# 6. Move them into a Training and Testing Partition

# OPTIONAL - 7. Compress them for Colab Training

In [9]:
TRAIN_PATH = os.path.join(ROOT_DIR, 'workspace', 'images', 'train')
TEST_PATH = os.path.join(ROOT_DIR, 'workspace', 'images', 'test')
#make these folders
if not os.path.exists(TRAIN_PATH):
    if os.name == 'posix':
        !mkdir -p {TRAIN_PATH}
    if os.name == 'nt':
        !mkdir {TRAIN_PATH}
if not os.path.exists(TEST_PATH):
    if os.name == 'posix':
        !mkdir -p {TEST_PATH}
    if os.name == 'nt':
        !mkdir {TEST_PATH}

In [ ]:
# !tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}